# mnist cnn

## imports

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## helpers

### weight initialization

In [4]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape=shape, stddev=0.1)
    return tf.Variable(initial_value=init_random_dist)

### bias initialization

In [5]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

### 2d convolution layer

In [37]:
padding = "SAME"

In [38]:
def conv2d(x, w):
    # x is features/data, is of shape [batch_size, image_height, image_width, num_color_channels]
    # w is the kernal (convolution window),
        #is of shape [filter_height, filter_width, num_channels_in, num_channels_out]
    return tf.nn.conv2d(input=x, filter=w, strides=[1]*4, padding=padding)

### pooling layer

In [39]:
def max_pool_2by2(x):
    #x i feature matrix, of shape [batch_size, image_height, image_width, num_color_channels]
    return tf.nn.max_pool(value=x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding=padding)

## layers

### convolutional

In [40]:
def convolutional_layer(input_x, shape):
    w = init_weights(shape=shape)
    b = init_bias(shape=[shape[3]])
    return tf.nn.relu(
        conv2d(x=input_x, w=w) + b
    )

### densely connected

In [41]:
def densely_connected_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    w = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, w) + b

## build network

In [42]:
pixels = mnist.train.images[0].shape[0]
pixels

784

In [43]:
image_dim = int(pixels**.5)
image_dim

28

In [44]:
num_classes = mnist.train.labels[0].shape[0]
num_classes

10

### placeholders

In [45]:
mnist.train.labels[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.])

In [46]:
x = tf.placeholder(dtype=tf.float32, shape=[None, pixels])
y = tf.placeholder(dtype=tf.float32, shape=[None, num_classes])

### layers

#### input layer
No idea what the [-1](https://www.tensorflow.org/api_docs/python/tf/reshape) is, but the 2nd and 3rd elements are image dimensions, and the 4th is color channels (1 for black and white).

In [47]:
input_layer = tf.reshape(x, [-1, image_dim, image_dim, 1]) 

#### conv1 layer
32 features for each 5x5x1 convolution window.

In [48]:
conv1 = convolutional_layer(input_x=input_layer, shape=[5, 5, 1, 32])

#### pool1 layer

In [49]:
pool1 = max_pool_2by2(x=conv1)

#### conv2

In [50]:
conv2 = convolutional_layer(input_x=pool1, shape=[5, 5, 32, 64])

#### pool2

In [51]:
pool2 = max_pool_2by2(x=conv2)

#### layer3_flat
Max pooling layer is 2x2, so 28x28 -> 14x14 (first maxpool) -> 7x7 (2nd max pool)

In [55]:
layer3_flat = tf.nn.relu(
    densely_connected_layer(
        input_layer=tf.reshape(pool2, shape=[-1, 7*7*64]),
        size=1024
    )
)

#### dropout

In [56]:
keep_prob = tf.placeholder(tf.float32)
layer4_dropout = tf.nn.dropout(x=layer3_flat, keep_prob=keep_prob,)